In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
CEREBRAS_API_KEY=os.getenv('CEREBRAS_API_KEY')
TAVILY_API_KEY=os.getenv('TAVILY_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
LANGSMITH_TRACING=os.getenv('LANGSMITH_TRACING')
LANGSMITH_ENDPOINT=os.getenv('LANGSMITH_ENDPOINT')
LANGSMITH_API_KEY=os.getenv('LANGSMITH_API_KEY')
LANGSMITH_PROJECT=os.getenv('LANGSMITH_PROJECT')
TAVILY_API_KEY=os.getenv('TAVILY_API_KEY')

In [3]:
os.environ['CEREBRAS_API_KEY'] = CEREBRAS_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
os.environ['LANGSMITH_API_KEY'] = LANGSMITH_API_KEY
os.environ['LANGSMITH_TRACING'] = "true"
os.environ['LANGSMITH_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGSMITH_PROJECT'] = LANGSMITH_PROJECT
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [4]:
from langchain_cerebras import ChatCerebras

llm = ChatCerebras(
    model="llama-3.3-70b",
)

In [5]:
# from sentence_transformers import SentenceTransformer
# embedding = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

from langchain_huggingface import HuggingFaceEmbeddings

# Wrapping the SentenceTransformer with LangChain-compatible embedding
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

# Predefine Tools

In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [8]:
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
print(tool.run({'query':'langchain'}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that enables generative artificial intelligence (Gen AI) models to retrieve and incorporate new information. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to supplement information from its pre-existing training data. This allows LLMs to use domain-specific and/or updated information. Use cases include providing chatbot access to internal company data or generating responses based on authoritative sources.
RAG improves la

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()
tool.invoke({'query': 'Who is Ochi Ojie'})

[{'title': 'Ochi | Wookieepedia - Fandom',
  'url': 'https://starwars.fandom.com/wiki/Ochi',
  'content': "Ochi was a Bestoonian male assassin and cultist of the Sith known as Ochi of Bestoon, hailing from the world of the same name with a career spanning decades, from the twilight of the Galactic Republic, and the subsequent rise of the Galactic Empire to the era of the New Republic. Ochi possessed a personal Sith dagger used to commit acts of murder, and had head implants. Trained in combat and martial skills, Ochi was a very capable fighter and lent his services to the Sith Lords Darth Sidious [...] During the Imperial Era, Ochi was an associate of advisor Yupe Tashu and was employed as an assassin by both the Emperor, Darth Sidious, and his right-hand, Darth Vader. Ochi also took part in the auction of Han Solo and became an undercover operative of Lady Qi'ra and Crimson Dawn following the Battle of Hoth at the height of the Galactic Civil War. [...] By this time, Ochi's bones had 

# Create a custom tool

In [12]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word"""
    return len(word)

In [13]:
get_word_length.invoke("abc")

3

In [14]:
get_word_length.invoke("bunchiochi")

10

In [16]:
@tool
def multiply(a: int, b: int) ->int:
    """Multiple two numbers"""
    return a * b


@tool
def summation(a: int, b: int) ->int:
    """Multiple two numbers"""
    return a + b

In [18]:
multiply.description

'Multiple two numbers'

In [21]:
multiply.invoke({"a":20, "b":2})

40

In [22]:
summation.invoke({"a":20, "b":2})

22

# Concept of Agents

In [24]:
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [25]:
tool=load_tools(["wikipedia"], llm=llm)

In [27]:
agent = initialize_agent(tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

/var/folders/_f/t1w94w1n759_mq486wwt1z9m0000gn/T/ipykernel_64649/2451433261.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [29]:
agent.run("what is Elon Musk")



> Entering new AgentExecutor chain...
Thought: To answer the question "what is Elon Musk", I need to find information about Elon Musk, so I should search for him on Wikipedia.

Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman known for his leadership of Tesla, SpaceX, and X (formerly Twitter). Since 2025, he has been a senior advisor to United States president Donald Trump and the de facto head of the Department of Government Efficiency (DOGE). Musk has been considered the wealthiest person in the world since 2021; as of May 2025, Forbes estimates his net worth to be US$424.7 billion. He was named Time magazine's Person of the Year in 2021.
Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada. He graduated from the University of Pennsylvania in the U.S. before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2.

"Elon Musk is a businessman and entrepreneur known for his leadership of companies such as Tesla, SpaceX, and X (formerly Twitter). He is considered the wealthiest person in the world, with a net worth of over $424 billion. Musk has been involved in various business ventures, including the development of reusable rockets, electric vehicles, and artificial intelligence research. He has also been a prominent figure in politics, having supported Donald Trump and been appointed as the head of the Department of Government Efficiency (DOGE). Musk's views have been characterized as libertarian and far-right, and he has been criticized for making unscientific and misleading statements, as well as promoting conspiracy theories."